In [182]:
import os
import xml.etree.ElementTree
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.tokenize import wordpunct_tokenize
import string
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
#nltk.download()

In [ ]:
# Se escapearon los & en el archivo y se agregaron los tags <DOCS> para poder parsearlo como un xml
inputXML = './data/ap/ap.txt'
e = xml.etree.ElementTree.parse(inputXML).getroot()
docs = {}

for elem in e.getchildren():
    text = elem.find('TEXT').text
    docno = elem.find('DOCNO').text.strip()
    docs[docno] = text

#docs

In [ ]:
def get_wordnet_pos(tag):

    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english') +  list(string.punctuation)
vocabulary = {}
bigrams_map = {}

for docno, text in docs.items():
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        
        for bigram in bigrams(tokens):
            if (bigram[0] not in stop_words) and (bigram[1] not in stop_words):
                bigrams_map.setdefault((bigram[0], bigram[1]), 0)
                bigrams_map.setdefault((bigram[1], bigram[0]), 0)
                bigrams_map[bigram[0],bigram[1]] = bigrams_map[bigram[0],bigram[1]] + 1
                bigrams_map[bigram[1],bigram[0]] = bigrams_map[bigram[1],bigram[0]] + 1
        tags = nltk.pos_tag(tokens)
        for word, tag in tags:
            wornet_pos = get_wordnet_pos(tag)
            if wornet_pos:
                lemma = lemmatizer.lemmatize(word, wornet_pos)
            else:
                lema = lemmatizer.lemmatize(word)
            if lemma not in stop_words:
                vocabulary.setdefault(lemma, 0)
                vocabulary[lemma] = vocabulary[lemma] + 1


In [ ]:
print("El tamaño del vocabulario es", len(vocabulary))

In [ ]:
first_500 = list(map(lambda x: x[0], sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)[:500]))
ordered_bigrams = sorted(bigrams_map.items(), key=lambda x: x[1], reverse=True)

for word in first_500:
    for bigram_entry in ordered_bigrams:
        if bigram_entry[0][0] == word:
            print(word, ",", bigram_entry[0][1])
            break


## 2)

In [107]:
punctuation = list(string.punctuation)
tokens = []
with open('./data/on-the-origin-of-species.txt') as file:  
    data = file.read() 
    tokens = list(filter(lambda x: x not in punctuation, nltk.word_tokenize(data)))
    
tokens

['\ufeffProduced',
 'by',
 'Sue',
 'Asscher',
 'THE',
 'ORIGIN',
 'OF',
 'SPECIES',
 'BY',
 'MEANS',
 'OF',
 'NATURAL',
 'SELECTION',
 'or',
 'the',
 'PRESERVATION',
 'OF',
 'FAVOURED',
 'RACES',
 'IN',
 'THE',
 'STRUGGLE',
 'FOR',
 'LIFE',
 'By',
 'Charles',
 'Darwin',
 'M.A.',
 'F.R.S.',
 'Author',
 'of',
 '``',
 'The',
 'Descent',
 'of',
 'Man',
 "''",
 'etc.',
 'etc',
 'Sixth',
 'London',
 'Edition',
 'with',
 'all',
 'Additions',
 'and',
 'Corrections',
 'The',
 '6th',
 'Edition',
 'is',
 'often',
 'considered',
 'the',
 'definitive',
 'edition',
 'Also',
 'see',
 'Project',
 'Gutenberg',
 'Etext',
 '1228',
 'for',
 'the',
 'First',
 'Edition',
 '``',
 'But',
 'with',
 'regard',
 'to',
 'the',
 'material',
 'world',
 'we',
 'can',
 'at',
 'least',
 'go',
 'so',
 'far',
 'as',
 'this',
 '--',
 'we',
 'can',
 'perceive',
 'that',
 'events',
 'are',
 'brought',
 'about',
 'not',
 'by',
 'insulated',
 'interpositions',
 'of',
 'Divine',
 'power',
 'exerted',
 'in',
 'each',
 'particul

In [116]:
def generate_apparitions_vector(word):
    return [1 if x == word else 0 for x in tokens]

apparitions = generate_apparitions_vector('plants')


In [177]:
std = 50
std_square = std ** 2
twice_std_square = 2 * std_square
denominator = np.sqrt(2 * np.pi) * std

def gaussian(x):
    x_square = x ** 2 
    exp = (-1) * (x_square / twice_std_square) 
    numerator = np.e ** exp
    return numerator / denominator


In [183]:
text_size = len(tokens)

def ro_w(t, apparitions):
    sum = 0
    for i, v in enumerate(apparitions):
        if v != 0:
            sum += gaussian(t - i)
    return sum

a = [ro_w(i, apparitions) for i in range(text_size)]
plt.plot(a)

KeyboardInterrupt: 